In [ ]:
# 공통 환경 설정

from dotenv import load_dotenv
import os

# 작업디렉토리를 상위경로로 변경
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)

# 환경변수 설정
load_dotenv(dotenv_path=".env.testcase", override=True)

from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB


session = SessionLocal()
faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')

In [ ]:
# 예시 쿼리와 타겟 리스트
from app.process.reranker import AlfredReranker

query = "public Optional<SensorDataDto> getPigVital(Long vitalCode, Long pigCode)"

# flash_rank_rerank 함수 호출
reranker = AlfredReranker()
results = reranker.cross_encoder(query, faissVectorDB, k=5)

# 결과 출력
for result in results:
    print(f"Chunk: {result}")

In [ ]:
from app.chain_graph.agent_state import CodeAssistState
from app.chain_graph.code_assist_chain import CodeAssistChain
from app.db_model.database import SessionLocal
from app.vectordb.faiss_vectordb import FaissVectorDB
# session = SessionLocal()
# faissVectorDB = FaissVectorDB(db_session=session, index_name='cg_code_assist')    
    
state = CodeAssistState()
state["question"] = "스페인의 비는 어디에 내리나요?"

code_assist_chain = CodeAssistChain()
code_assist_state = code_assist_chain.context_node(state=state, k=3)